In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("data.csv")

In [42]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [43]:
df.shape

(169909, 19)

In [44]:
df.columns

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year'],
      dtype='object')

In [45]:
newPop = []
for i in df['popularity']:
    newPop.append(i/100)
df['popularity'] = newPop

In [46]:
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
%matplotlib inline
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [47]:
X = df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo']]
y = df[['popularity']]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

In [49]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([118936, 9]) torch.Size([118936])
torch.Size([50973, 9]) torch.Size([50973])


In [50]:
class Net(nn.Module):
  def __init__(self,n_features):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_features, 5)
    self.fc2 = nn.Linear(5, 3)
    self.fc3 = nn.Linear(3, 1)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return torch.sigmoid(self.fc3(x))

In [51]:
net = Net(X_train.shape[1])

In [52]:
X_train.shape[1]

9

In [53]:
criterion = nn.BCELoss()

In [54]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [55]:
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

In [56]:
def round_tensor(t, decimal_places=2):
  return round(t.item(), decimal_places)
for epoch in range(1000):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 100 == 0:
        train_acc = calculate_accuracy(y_train, y_pred)
        y_test_pred = net(X_test)
        y_test_pred = torch.squeeze(y_test_pred)
        test_loss = criterion(y_test_pred, y_test)
        test_acc = calculate_accuracy(y_test, y_test_pred)
        print(f'''epoch {epoch}Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
''')
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

epoch 0Train set - loss: 0.73, accuracy: 0.0
Test  set - loss: 0.73, accuracy: 0.0

epoch 100Train set - loss: 0.71, accuracy: 0.0
Test  set - loss: 0.71, accuracy: 0.0

epoch 200Train set - loss: 0.69, accuracy: 0.0
Test  set - loss: 0.69, accuracy: 0.0

epoch 300Train set - loss: 0.68, accuracy: 0.0
Test  set - loss: 0.68, accuracy: 0.0

epoch 400Train set - loss: 0.67, accuracy: 0.0
Test  set - loss: 0.67, accuracy: 0.0

epoch 500Train set - loss: 0.66, accuracy: 0.0
Test  set - loss: 0.66, accuracy: 0.0

epoch 600Train set - loss: 0.65, accuracy: 0.0
Test  set - loss: 0.65, accuracy: 0.0

epoch 700Train set - loss: 0.64, accuracy: 0.0
Test  set - loss: 0.64, accuracy: 0.0

epoch 800Train set - loss: 0.64, accuracy: 0.0
Test  set - loss: 0.64, accuracy: 0.0

epoch 900Train set - loss: 0.63, accuracy: 0.0
Test  set - loss: 0.63, accuracy: 0.0



In [57]:
MODEL_PATH = 'model.pth'
torch.save(net, MODEL_PATH)

In [58]:
net = torch.load(MODEL_PATH)

In [59]:
df['popularity'].min()

0.0

In [60]:
y_pred[0]

tensor(0.3701, grad_fn=<SelectBackward>)

In [61]:
y_train[0]

tensor(0.4200)

In [62]:
abs(y_pred[0] - y_train[0])

tensor(0.0499, grad_fn=<AbsBackward>)

In [63]:
test_loss = criterion(y_test_pred, y_test)

In [64]:
test_loss

tensor(0.6330, grad_fn=<BinaryCrossEntropyBackward>)

In [65]:
count = 0
for i in range(len(y_pred)):
    if (abs(y_pred[i] - y_train[i]) < 0.1):
        count = count + 1

In [66]:
count/len(y_train)

0.3349196206363086

In [67]:
count = 0
for i in range(len(y_test)):
    if (abs(y_test_pred[i] - y_test[i]) < 0.1):
        count = count + 1

In [68]:
count/len(y_test)

0.3377474349165244